In [321]:
# imports 
import os
import json
import operator

# Players
BotHash = "FDDD772696374FE2C5F932BB598533D9"
SB = 10
Players = [
   {
        'Name': 'Bold',
        'games': 'Bold'
    },
    {
        'Name': 'FoldMan',
        'games': 'FoldMan'
    },
    {
        'Name': 'Fish',
       'games': 'Fish'
    },
     {
        'Name': 'Random',
        'games': 'Random',
    },
    {
        'Name': 'HonestMan',
        'games':'HonestMan',
    }
]

def prettyPrint(*arguments):
    out = ''
    for arg in arguments:
        space = (30-len(arg)) * " "
        out += arg + space
    print(out)
    
# Load file paths of player games.
def LoadPlayerGames(player):
    #return [os.path.join(r,file) for r,d,f in os.walk(player['games']) for file in f]
    return [os.path.join(player['games'],f) for f in os.listdir(player['games']) if not f.startswith('.')]

# Load game as json object.
def LoadGame(path):
    with open(path) as f:
        g = json.load(f)
        if g["BotHash"] != BotHash:
            raise Exception("BotHash does not match: " + g["BotHash"])
        return g

# Get player result from game (W,D,L)
def GetGameResultForPlayer(game, playerName):
    results = game["Results"]
    maxStack = 0
    hasMax = []
    # find max stack
    for result in results:
        if maxStack < result["Stack"]:
            maxStack = result["Stack"]
            hasMax= [result["Player"]["Name"]]
        elif maxStack == result["Stack"]:
            hasMax.append(result["Player"]["Name"])
    # find player result
    if playerName in hasMax:
        if len(hasMax) > 1:
            return 'D'
        else:
            return 'W'
    return 'L'
            
# Get player results against opponent
def GetPlayerGamesResults(playerName, opponent):
    won = 0
    lost = 0
    drew = 0
    files = LoadPlayerGames(opponent)
    for file in files:
        game = LoadGame(file)
        result = GetGameResultForPlayer(game, playerName)
        if result == 'W':
            won += 1
        elif result == 'D':
            drew += 1
        elif result == 'L':
            lost += 1
        else:
            raise Exception("Error :(")
    return (won, drew, lost)

# Get player result from round (W,D,L)
def GetRoundResultForPlayer(r, playerName):
    results = r["Winner"]
    winners = len(results)
    won = False
    for result in results:
        if playerName == result["Name"]:
            won = True
            break
    if won:
        if winners > 1:
            return 'D'
        else:
            return 'W'
    return 'L'

# Get player round results from game
def GetRoundResultsForPlayer(game, playerName):
    won = 0
    lost = 0
    drew = 0
    for r in game["Rounds"]:
        result = GetRoundResultForPlayer(r, playerName)
        if result == 'W':
            won += 1
        elif result == 'D':
            drew += 1
        elif result == 'L':
            lost += 1
        else:
            raise Exception("Error :(")
    return (won, drew, lost)

# Get player round results against an opponent
def GetPlayerRoundResults(playerName, opponent):
    won = 0
    lost = 0
    drew = 0
    files = LoadPlayerGames(opponent)
    for file in files:
        game = LoadGame(file)
        w,d,l = GetRoundResultsForPlayer(game, playerName)
        won += w
        drew += d
        lost += l
    return (won, drew, lost)

def GetPlayerPlayingStyle(opponent,playerName):
    playerModel = {
        'LooseAggressive': 0,
        'LoosePassive': 0,
        'TightAggressive': 0,
        'TightPassive': 0,
        'Balanced' : 0,
    }
    opponentModel = {
        'LooseAggressive': 0,
        'LoosePassive': 0,
        'TightAggressive': 0,
        'TightPassive': 0,
        'Balanced' : 0,
    }
    
    files = LoadPlayerGames(opponent)
    for file in files:
        game = LoadGame(file)
        models = game["PlayerModels"]
        for m in models:
            if m["Player"]["Name"] == playerName:
                playerModel[m["PlayingStyle"]] +=1
            if m["Player"]["Name"] == opponent["Name"]:
                opponentModel[m["PlayingStyle"]] +=1
    return (max(playerModel.items(), key=operator.itemgetter(1))[0], max(opponentModel.items(), key=operator.itemgetter(1))[0])

    
def GetWPH(playerName, opponent, rn):
    stackSum = 0
    initStack = 0
    files = LoadPlayerGames(opponent)
    for file in files:
        game = LoadGame(file)
        for r in game["Results"]:
            if playerName == r["Player"]["Name"]:
                stackSum += r["Stack"]
                initStack += r["InitialStack"]
                break
    return (stackSum-initStack)/rn


def GetFWPHForPlayer(game, playerName):
    loss = 0
    rn = len(game["Rounds"])
    for r in game["Rounds"]:
        sbp = r["SmallBlindPosition"]
        bbp = r["BigBlindPosition"]
        if r["Players"][sbp]["Name"] == playerName:
            loss -= SB
        if r["Players"][bbp]["Name"] == playerName:
            loss -= SB * 2
    return loss/rn

def GetFWPH(playerName, opponent):
    files = LoadPlayerGames(opponent)
    fwphSum = 0
    gn = len(files)
    for file in files:
        game = LoadGame(file)
        fwphSum += GetFWPHForPlayer(game, playerName)
    return fwphSum/gn   

def AdjustedWPH(wph, fwph):
    return wph - fwph
        
def PrintResultOf(playerName):
    for player in Players:
        prettyPrint("Results against:", player["Name"])
        n = len(LoadPlayerGames(player))
        w,d,l = GetPlayerGamesResults(playerName, player)
        # game states
        prettyPrint("number of games:",str(n))
        prettyPrint("number of games won:", str(w) , "{:.1f}".format(w*100/n) + "%")
        prettyPrint("number of games drew:", str(d), "{:.1f}".format(d*100/n) + "%")
        prettyPrint("number of games lost:", str(l), "{:.1f}".format(l*100/n) + "%")
        # round states
        rw,rd,rl = GetPlayerRoundResults(playerName, player)
        rn = rw+rd+rl
        prettyPrint("number of rounds:",str(rn))
        prettyPrint("number of rounds won:", str(rw) , "{:.1f}".format(rw*100/rn) + "%")
        prettyPrint("number of rounds drew:", str(rd), "{:.1f}".format(rd*100/rn) + "%")
        prettyPrint("number of rounds lost:", str(rl), "{:.1f}".format(rl*100/rn) + "%")
        # wph
        wph = GetWPH(playerName, player, rn)
        fwph = GetFWPH(playerName, player)
        prettyPrint("WPH:", "{:.1f}".format(wph))
        prettyPrint("FWPH:", "{:.1f}".format(fwph))
        prettyPrint("AWPH:", "{:.1f}".format(AdjustedWPH(wph, fwph)))
        # playing style
        ps, os = GetPlayerPlayingStyle(player,playerName)
        prettyPrint(playerName + " Playing Style:", ps)
        prettyPrint(player["Name"] + " Playing Style:", os)
        print('-------------------------------------------------------------')

In [322]:
PrintResultOf('PokerShark')

Results against:              Bold                          
number of games:              100                           
number of games won:          47                            47.0%                         
number of games drew:         0                             0.0%                          
number of games lost:         53                            53.0%                         
number of rounds:             632                           
number of rounds won:         117                           18.5%                         
number of rounds drew:        0                             0.0%                          
number of rounds lost:        515                           81.5%                         
WPH:                          -9.5                          
FWPH:                         -15.6                         
AWPH:                         6.1                           
PokerShark Playing Style:     TightPassive                  
Bold Playing Style:        